In [2]:
import random
import csv

# S-Box: Used for substitution operation, providing a non-linear substitution step.
S_Box = [0xE, 0x4, 0xD, 0x1, 0x2, 0xF, 0xB, 0x8, 0x3, 0xA, 0x6, 0xC, 0x5, 0x9, 0x0, 0x7]

# P-Box: Used for permutation operation, it rearranges the input bits.
P_Box = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]


# Generate five 16-bit subkeys from a 32-bit master key.
def gen_K_list(K):
    Ks = []
    for _ in range(5):
        Ks.append(K & 0xFFFF)  # Get the last 16 bits of K
        K >>= 4  # Right shift by 4 bits
    return Ks[::-1]

# Perform substitution operation using the S-Box.
def pi_s(s_box, ur):
    vr = 0
    for i in range(4):
        uri = ur & 0xF  # Get the last 4 bits of ur
        vri = s_box[uri]  # Get the substitute value from the S-Box
        vr += vri << (4 * i)  # Store the result in vr
        ur >>= 4  # Right shift by 4 bits
    return vr

# Perform permutation operation using the P-Box.
def pi_p(p_box, vr):
    wr = 0
    for i in range(15, -1, -1):
        vri = vr & 1  # Get the last bit of vr
        vr >>= 1  # Right shift by 1 bit
        wr |= vri << (16 - p_box[i])  # Rearrange bits based on P-Box value
    return wr

# Implement the SPN encryption operation.
def do_SPN(x, s_box, p_box, Ks):
    wr = x
    # Execute Nr-1 rounds of encryption, each round includes three steps: Key addition, substitution, and permutation.
    for r in range(3):
        ur = wr ^ Ks[r]  # Key addition
        vr = pi_s(s_box, ur)  # Substitution
        wr = pi_p(p_box, vr)  # Permutation
    # The last round does not include permutation.
    ur = wr ^ Ks[3]  # Key addition
    vr = pi_s(s_box, ur)  # Substitution
    return vr ^ Ks[4]  # Output the result after another key addition

# Encrypt a 16-bit plaintext with a given 32-bit key.
def encrypt(K, x):
    Ks = gen_K_list(K)  # Generate subkeys
    return do_SPN(x, S_Box, P_Box, Ks)  # Execute SPN encryption

# Generate plaintext-ciphertext pairs
def generate_pairs():
    K = int("00111010100101001101011000111111", 2)
    
    pairs = []
    for _ in range(20):
        plaintext = random.randint(0, 0xFFFF)
        ciphertext = encrypt(K, plaintext)
        pairs.append((format(plaintext, '016b'), format(ciphertext, '016b')))
    
    return pairs

# Save the pairs to a CSV file
def save_to_csv(pairs, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Plaintext', 'Ciphertext']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for p, c in pairs:
            writer.writerow({'Plaintext': p, 'Ciphertext': c})


pairs = generate_pairs()
filename = "plaintext_ciphertext_pairs.csv"
save_to_csv(pairs, filename)
